In [206]:
import pandas as pd
import numpy as np 
import imageio as iio
import matplotlib.pyplot as plt
from keras.models import Sequential 
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [207]:

MAX_PICTURES = 100

In [208]:
df = pd.read_csv('merged.csv', index_col=0)

In [209]:
df = df.sample(frac=1).reset_index(drop=True)

In [210]:
# read first 1000 images from dataframe 
image_list = list()
for image in df['image_path'][0:MAX_PICTURES]:
    im = iio.imread(image)
    image_list.append(im)


In [211]:
mel_dict = {
    'akiec': 'Actinic keratosis-like lesions',
    'bcc': 'basal cell carcinoma',
    'bkl': 'benign keratosis',
    'df': 'dermatofibroma',
    'mel': 'melanoma',
    'nv': 'melanocytic nevi',
    'vasc': 'vascular lesions / hemorrhage'
}

In [212]:
# should be random every time
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path
7627,HAM_0005190,ISIC_0032700,nv,histo,40.0,female,back,data/ham10000_images_part_2/ISIC_0032700.jpg
2967,HAM_0004685,ISIC_0030767,bcc,histo,75.0,male,scalp,data/ham10000_images_part_2/ISIC_0030767.jpg
9262,HAM_0005531,ISIC_0024577,nv,consensus,45.0,male,lower extremity,data/ham10000_images_part_1/ISIC_0024577.jpg
1528,HAM_0005368,ISIC_0034087,mel,histo,45.0,male,chest,data/ham10000_images_part_2/ISIC_0034087.jpg
913,HAM_0003787,ISIC_0030630,bkl,consensus,70.0,male,chest,data/ham10000_images_part_2/ISIC_0030630.jpg


In [213]:
SPLIT = MAX_PICTURES / 0.80 
SPLIT = round(SPLIT)
x_train = image_array.reshape(MAX_PICTURES,450,600,3)
x_test = x_train[SPLIT:MAX_PICTURES]
x_train = x_train[0:SPLIT]

y = df['dx'][0:MAX_PICTURES].values

y_train = y[0:SPLIT]
y_test = y_train[SPLIT:MAX_PICTURES]

x_train = x_train / 255
x_test = x_test / 255

In [214]:


model = Sequential()
model.add(Input(shape=(450,600,3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(7))

y_encoded = LabelEncoder()
y_train = y_encoded.fit_transform(y_train)

In [215]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [216]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 448, 598, 32)      896       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 224, 299, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_31 (Conv2D)          (None, 222, 297, 64)      18496     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 111, 148, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_32 (Conv2D)          (None, 109, 146, 64)      36928     
                                                                 
 flatten_10 (Flatten)        (None, 1018496)         

In [219]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
4/4 [==============================] - 11s 2s/step - loss: 12.3060 - accuracy: 0.0200
Epoch 2/10
4/4 [==============================] - 11s 2s/step - loss: 0.0699 - accuracy: 0.0100
Epoch 3/10
4/4 [==============================] - 11s 3s/step - loss: 0.0699 - accuracy: 0.0100
Epoch 4/10
4/4 [==============================] - 11s 3s/step - loss: 0.0699 - accuracy: 0.0100
Epoch 5/10
4/4 [==============================] - 11s 2s/step - loss: 0.0699 - accuracy: 0.0100
Epoch 6/10
4/4 [==============================] - 11s 3s/step - loss: 0.0699 - accuracy: 0.0100
Epoch 7/10
4/4 [==============================] - 11s 3s/step - loss: 0.0699 - accuracy: 0.0100
Epoch 8/10
4/4 [==============================] - 12s 3s/step - loss: 0.0699 - accuracy: 0.0100
Epoch 9/10
4/4 [==============================] - 11s 2s/step - loss: 0.0699 - accuracy: 0.0100
Epoch 10/10
4/4 [==============================] - 11s 3s/step - loss: 0.0699 - accuracy: 0.0100
